# Import Bibliotheken

In [5]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

### Package Versionen

In [6]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [7]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [8]:
comment_df = pd.read_excel('../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [9]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [10]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [11]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

## Dataloader

### Tokenisierung

In [12]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

### Erstellung Vokabular

In [13]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [14]:
# Länge des Trainungsets
len(train_set)

2798

In [15]:
tokens = yield_tokens(train_set)

In [16]:
next(iter(tokens))

['nettes',
 'personal',
 'tolle',
 'café',
 'ecke',
 'wo',
 'man',
 'gemütlich',
 'auf',
 'sein',
 'auto',
 'warten',
 'kann',
 'wenn',
 'es',
 'zur',
 'reperatur',
 'ist',
 'auch',
 'für',
 'reisende',
 'ein',
 'guter',
 'platz',
 'zum',
 'auftanken',
 'für',
 'auto',
 'und',
 'mensch']

In [17]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [18]:
len(vocab)

2404

In [19]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [20]:
onehot = torch.zeros(1,len(vocab))

In [21]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [22]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [23]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [24]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [25]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [26]:
unflat = [[x] for x in flat_set]

In [27]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1, specials=["<unk>"])

label_vocab.set_default_index(vocab["<unk>"])

In [28]:
label_vocab.lookup_token(3)

'DigitalFueling'

#### Dictionary erstellen

In [29]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Erscheinungsbild': 0,
 'Personal': 1,
 'Sanitär': 2,
 'Paketservice': 3,
 'Tankpool': 4,
 'E-Mobilität': 5,
 'E-Fuels': 6,
 'Pricing': 7,
 'Kartenakzeptanzen': 8,
 'Luft': 9,
 'Bistro': 10,
 'Nacht-/Tankautomat': 11,
 'WLAN': 12,
 'AdBlue': 13,
 'Waschanlage': 14,
 'Staubsauger': 15,
 'Öffnungszeiten': 16,
 'Verkehrsanbindung': 17,
 'SB-Waschboxen': 18,
 'DigitalFueling': 19,
 'Shop': 20,
 'Kraftstoffauswahl': 21,
 'Werkstatt': 22,
 'Zapfsäulen': 23}

### Erstellung Dataloader

In [30]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [31]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=collate_batch, num_workers=0)

## Architektur und Training

### Architektur

In [32]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [33]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [34]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.MSELoss()

## Initialisierung Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Definition der Epochen
num_epochs = 250

In [35]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

loss_hist = {}
accuracy_hist = {}

In [36]:
for (text, label) in train_loader:
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        1])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


In [37]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().numpy()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.31236597895622253
Training Accuracy for epoch 1:0.383130818605423
Training Accuracy for epoch 2:0.39385274052619934
Training Accuracy for epoch 3:0.3877769708633423
Training Accuracy for epoch 4:0.3738384544849396
Training Accuracy for epoch 5:0.3834882080554962
Training Accuracy for epoch 6:0.3806290328502655
Training Accuracy for epoch 7:0.35596853494644165
Training Accuracy for epoch 8:0.372051477432251
Training Accuracy for epoch 9:0.3677626848220825
Training Accuracy for epoch 10:0.383130818605423
Training Accuracy for epoch 11:0.38741958141326904
Training Accuracy for epoch 12:0.403502494096756
Training Accuracy for epoch 13:0.40314510464668274
Training Accuracy for epoch 14:0.40135812759399414
Training Accuracy for epoch 15:0.4045746922492981
Training Accuracy for epoch 16:0.3956397473812103
Training Accuracy for epoch 17:0.38920658826828003
Training Accuracy for epoch 18:0.39492493867874146
Training Accuracy for epoch 19:0.3917083740234375
Traini

In [38]:
total_train_accuracy = train_accuracy.compute()

c:\Users\flori\anaconda3\envs\ki_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: The ``compute`` method of metric Accuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


### Auswertung Fehlerentwicklung

In [39]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.060781
1,1,0.060652
2,2,0.063311
3,3,0.064778
4,4,0.064712


In [40]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
)
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [41]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.312366
1,1,0.383131
2,2,0.393853
3,3,0.387777
4,4,0.373838


In [42]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
)
accuracy_chart

alt.Chart(...)

## Evaluierung

In [43]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2404, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=25, bias=True)
)

### Gesamtevaluierung

In [44]:
test_loader = DataLoader(test_set, batch_size= 16, shuffle=True,collate_fn=collate_batch)

In [45]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [46]:
total_valid_accuracy

tensor(0.4312)

### Evaluierung nach Klassen

In [47]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=25
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [48]:
total_valid_accuracy

tensor([0.5279, 0.9925, 0.8299, 0.9908,    nan, 0.9983, 0.8824, 0.9683, 0.9450,
        0.9808, 0.9783, 0.9892, 0.8874, 0.8082, 0.9867, 0.9641, 0.8599, 0.9933,
        0.9992, 0.9691,    nan, 0.9016, 0.9908, 0.9892, 0.9700])

### Evaluierung nach der Anzahl der Epochen

In [49]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=collate_batch)

In [50]:
test_loader = DataLoader(test_set, batch_size=16, shuffle=True, collate_fn=collate_batch)

In [51]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [52]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.MSELoss()

## Initialisierung Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Definition der Epochen
num_epochs = 250

In [53]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [54]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_train_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = loss_fn(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()

Training Accuracy for epoch 0: 0.32273051142692566
Validation Accuracy for epoch 0: 0.3828190267086029
Training Accuracy for epoch 1: 0.38920658826828003
Validation Accuracy for epoch 1: 0.43119266629219055
Training Accuracy for epoch 2: 0.39957112073898315
Validation Accuracy for epoch 2: 0.4128440320491791
Training Accuracy for epoch 3: 0.3999285101890564
Validation Accuracy for epoch 3: 0.4203502833843231
Training Accuracy for epoch 4: 0.3945675492286682
Validation Accuracy for epoch 4: 0.38698914647102356
Training Accuracy for epoch 5: 0.38920658826828003
Validation Accuracy for epoch 5: 0.4153461158275604
Training Accuracy for epoch 6: 0.3877769708633423
Validation Accuracy for epoch 6: 0.36113429069519043
Training Accuracy for epoch 7: 0.37919941544532776
Validation Accuracy for epoch 7: 0.40200167894363403
Training Accuracy for epoch 8: 0.38563260436058044
Validation Accuracy for epoch 8: 0.39032527804374695
Training Accuracy for epoch 9: 0.3881343901157379
Validation Accuracy f

In [55]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "Train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "Test"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [56]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
    x = alt.X('Epoch',title='Anzahl Epochen'),
    y = alt.Y('Loss',title='Mittlerer Fehler'),
    color = alt.Color('Type',title='Typ')
)
train_eval_loss_chart

alt.Chart(...)

In [57]:
valid_accuracy_hist

{0: array(0.38281903, dtype=float32),
 1: array(0.43119267, dtype=float32),
 2: array(0.41284403, dtype=float32),
 3: array(0.42035028, dtype=float32),
 4: array(0.38698915, dtype=float32),
 5: array(0.41534612, dtype=float32),
 6: array(0.3611343, dtype=float32),
 7: array(0.40200168, dtype=float32),
 8: array(0.39032528, dtype=float32),
 9: array(0.41201, dtype=float32),
 10: array(0.41534612, dtype=float32),
 11: array(0.37781486, dtype=float32),
 12: array(0.34445372, dtype=float32),
 13: array(0.37114263, dtype=float32),
 14: array(0.39282736, dtype=float32),
 15: array(0.39449543, dtype=float32),
 16: array(0.3619683, dtype=float32),
 17: array(0.28940785, dtype=float32),
 18: array(0.33027524, dtype=float32),
 19: array(0.3411176, dtype=float32),
 20: array(0.34195164, dtype=float32),
 21: array(0.34862384, dtype=float32),
 22: array(0.3377815, dtype=float32),
 23: array(0.3294412, dtype=float32),
 24: array(0.30692244, dtype=float32),
 25: array(0.32527107, dtype=float32),
 26:

In [58]:
train_accuracy_hist

{0: array(0.3227305, dtype=float32),
 1: array(0.3892066, dtype=float32),
 2: array(0.39957112, dtype=float32),
 3: array(0.3999285, dtype=float32),
 4: array(0.39456755, dtype=float32),
 5: array(0.3892066, dtype=float32),
 6: array(0.38777697, dtype=float32),
 7: array(0.37919942, dtype=float32),
 8: array(0.3856326, dtype=float32),
 9: array(0.3881344, dtype=float32),
 10: array(0.39421016, dtype=float32),
 11: array(0.37919942, dtype=float32),
 12: array(0.3498928, dtype=float32),
 13: array(0.3638313, dtype=float32),
 14: array(0.3695497, dtype=float32),
 15: array(0.36776268, dtype=float32),
 16: array(0.36776268, dtype=float32),
 17: array(0.2691208, dtype=float32),
 18: array(0.32451752, dtype=float32),
 19: array(0.32094353, dtype=float32),
 20: array(0.33452466, dtype=float32),
 21: array(0.33559686, dtype=float32),
 22: array(0.34024304, dtype=float32),
 23: array(0.3252323, dtype=float32),
 24: array(0.31272337, dtype=float32),
 25: array(0.29092208, dtype=float32),
 26: ar

In [59]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [60]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "Train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "Test"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [61]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.389207,Train
1,2,0.399571,Train
2,3,0.399929,Train
3,4,0.394568,Train
4,5,0.389207,Train
...,...,...,...
244,245,0.402836,Test
245,246,0.402836,Test
246,247,0.402836,Test
247,248,0.402836,Test


In [62]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x = alt.X('Epoch',title='Anzahl Epochen'),
    y = alt.Y('Accuracy',title='Genauigkeit'),
    color = alt.Color('Type',title='Typ')
)
train_eval_accuracy_chart

alt.Chart(...)